In [19]:
import pandas as pd
import numpy as np
import re
import os

pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [20]:
pdfs = pd.read_csv('data-pdf.csv')
pdfs.head()

,pdf,teléfono,habitaciones,Equipment,precio_alquiler,superficie,baños,contacto
0,"Alquiler de piso en avenida de la Constitución, 5, San Miguel, Murcia.pdf",690 964 656,3,Amueblado,800.0,110.0,2.0,Elena
1,"Alquiler de piso en avenida de la Constitución, 5, San Miguel, Murcia.pdf",651 915 502,3,Amueblado,850.0,125.0,2.0,Marco
2,"Alquiler de piso en calle Riquelme, 14, Santa Catalina-San Bartolomé, Murcia.pdf",644 463 987,4,equipada,300.0,80.0,1.0,NaN
3,"Alquiler de piso en calle Cánovas del Castillo, 67, Santiago de la Ribera, San Javier.pdf",665 795 659,2,amueblado,800.0,82.0,2.0,Angel
4,"Alquiler de piso en calle urb veneziola, 3, Playa del Esparto-Veneziola, La Manga del Mar Menor.pdf",671 305 483,2,Amueblado,300.0,28.0,1.0,Alicia


In [21]:
pdfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pdf              1970 non-null   object 
 1   teléfono         1970 non-null   object 
 2   habitaciones     1970 non-null   object 
 3   Equipment        1695 non-null   object 
 4   precio_alquiler  1588 non-null   float64
 5   superficie       1586 non-null   float64
 6   baños            1721 non-null   float64
 7   contacto         1251 non-null   object 
dtypes: float64(3), object(5)
memory usage: 123.3+ KB


In [22]:
pdfs['type_operation'] = 'Alquiler'
pdfs['profile'] = 'Particular'
pdfs['date'] = '2018-01-01'
pdfs['precio_alquiler'] = pdfs['precio_alquiler'].fillna(0).astype(int)
pdfs['superficie'] = pdfs['superficie'].fillna(0).astype(int)
pdfs = pdfs.rename(columns={'contacto': 'name_contact', 
                            'pdf': 'source', 
                            'Equipment': 'features', 
                            'habitaciones': 'rooms',
                            'superficie': 'meters',
                            'precio_alquiler': 'price',
                            'teléfono': 'telephone'})


# Definir la función para extraer los datos
def extraer_datos(row):
    # Usar una expresión regular para capturar "tipo de vivienda", "street" y "place"
    match = re.search(r'Alquiler de (.+?) en (.*?, \d+), (.*?)\.pdf', row)
    if match:
        property = match.group(1)  # Captura el tipo de vivienda (piso, casa, etc.)
        street = match.group(2)         # Captura la calle y número
        place = match.group(3)          # Captura el barrio y la ciudad
        return pd.Series([property, street, place])
    else:
        return pd.Series([None, None, None])  # Si no coincide, devuelve None

# Aplicar la función al DataFrame y crear las nuevas columnas
pdfs[['tipo_vivienda', 'street', 'place']] = pdfs['source'].apply(extraer_datos)

In [25]:
euspay = pd.read_csv('euspay-murcia.csv')
euspay.drop("Unnamed: 0", axis=1, inplace=True)
euspay.head()

,id,name_contact,telephone,type_operation,property,profile,street,place,meters,rooms,orientation,features,price,source,id_source,url_source,date,date_scrape,ask_advertiser
0,1,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5","Puente Tocinos, Murcia",110,3,1ª planta exterior con ascensor,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,idealista,26861942,https://www.idealista.com/inmueble/26861942/,2018-12-01,2019-11-29,Todo
1,3,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41","Alquerías, Murcia",116,3,1ª planta exterior con ascensor Garaje incluido,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,idealista,83207117,https://www.idealista.com/inmueble/83207117/,2018-12-01,2019-11-29,Todo
2,8,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2","Espinardo, Murcia",95,3,5ª planta exterior con ascensor Garaje incluido,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,idealista,83714401,https://www.idealista.com/inmueble/83714401/,2019-01-01,2022-06-14,Todo
3,9,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14","Cabezo de Torres, Murcia",100,3,1ª planta exterior con ascensor Garaje incluido,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,idealista,83714094,https://www.idealista.com/inmueble/83714094/,2019-01-01,2019-11-29,Todo
4,12,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,Bajo exterior sin ascensor Garaje incluido,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,idealista,1675052,https://www.idealista.com/inmueble/1675052/,2019-01-01,2019-11-29,Todo


In [29]:
euspay.drop(columns=['source', 'id_source', 'date_scrape', 'ask_advertiser'], inplace=True)
euspay = euspay.rename(columns={'url_source': 'source'})

KeyError: "['source', 'id_source', 'date_scrape', 'ask_advertiser'] not found in axis"

In [30]:
euspay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11137 entries, 0 to 11136
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11137 non-null  int64 
 1   name_contact    11009 non-null  object
 2   telephone       10869 non-null  object
 3   type_operation  11137 non-null  object
 4   property        11137 non-null  object
 5   profile         11137 non-null  object
 6   street          9329 non-null   object
 7   place           11137 non-null  object
 8   meters          11137 non-null  int64 
 9   rooms           11137 non-null  object
 10  orientation     6050 non-null   object
 11  features        10621 non-null  object
 12  price           11137 non-null  int64 
 13  url_source      11137 non-null  object
 14  date            11137 non-null  object
dtypes: int64(3), object(12)
memory usage: 1.3+ MB


In [23]:
pdfs.isnull().sum()

source              0
telephone           0
rooms               0
features          275
price               0
meters              0
baños             249
name_contact      719
type_operation      0
profile             0
date                0
tipo_vivienda     325
street            325
place             325
dtype: int64

In [24]:
pdfs.sample(5)

,source,telephone,rooms,features,price,meters,baños,name_contact,type_operation,profile,date,tipo_vivienda,street,place
292,"Alquiler de piso en calle Cortes, 13, San Pedro, Murcia.pdf",691 590 987,No especificado,amueblado,0,0,1.0,OFERTA,Alquiler,Particular,2018-01-01,piso,"calle Cortes, 13","San Pedro, Murcia"
887,"MIL ANUNCIOS.COM - centro murcia ,Canovas del castillo Canovas del castillo en Murcia.pdf",No especificado,No especificado,amueblado,420,100,1.0,NaN,Alquiler,Particular,2018-01-01,None,None,None
389,"Alquiler de piso en plaza Cruz Roja, 11, San Juan, Murcia.pdf",600 603 166,No especificado,amueblado,590,80,1.0,Yolanda\nNota personal,Alquiler,Particular,2018-01-01,piso,"plaza Cruz Roja, 11","San Juan, Murcia"
1452,"Alquiler de Piso en calle Desamparados, 13, San Pedro, Murcia.pdf",616 928 014,3,Amueblado,750,110,2.0,C,Alquiler,Particular,2018-01-01,Piso,"calle Desamparados, 13","San Pedro, Murcia"
901,"Alquiler de chalet pareado en paseo Espejo, 153, Los Alcázares.pdf",968 512 494,2,amueblado,425,190,2.0,NaN,Alquiler,Particular,2018-01-01,chalet pareado,"paseo Espejo, 153",Los Alcázares
